# 1.Importing all libraries

importing all libaraties that are needed for data cleaning and visualization

In [89]:
import numpy as np
import pandas as pd
import seaborn as sb
import warnings
import json
import ast
warnings.filterwarnings('ignore')

Numpy used for handling statstical and numerical analysis

Pandas used for cleaning,updating and manipulating dataframes/datsets

Seaborn used for visualizing ratings

Warnings used for the clearing/ not displaying the warnings

In [90]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

reduce memory usage function is used to reduce the memory as such as unwanted memory such as int64 to int32 or float 64 to float32

In [91]:
credits=pd.read_csv('credits.csv') 
credits=reduce_memory_usage(credits)

Memory usage after optimization is: 0.87 MB
Decreased by 16.7%


credits dataset contains the attributes such as cast information, crew information and movie Id

In [92]:
keywords=pd.read_csv('keywords.csv')
keywords=reduce_memory_usage(keywords)

Memory usage after optimization is: 0.53 MB
Decreased by 25.0%


Keywords contains information about tags that are used for movies

In [93]:
links=pd.read_csv('links.csv')
links_small=pd.read_csv('links_small.csv')

In [94]:
links=reduce_memory_usage(links)
links_small=reduce_memory_usage(links_small)

Memory usage after optimization is: 0.52 MB
Decreased by 50.0%
Memory usage after optimization is: 0.10 MB
Decreased by 50.0%


Links contains the id and imdb id for all movies

In [95]:
ratings=pd.read_csv('ratings.csv')
ratings = reduce_memory_usage(ratings)

Memory usage after optimization is: 347.46 MB
Decreased by 56.2%


In [96]:
ratings_small=pd.read_csv('ratings_small.csv')
ratings_small= reduce_memory_usage(ratings_small)

Memory usage after optimization is: 1.14 MB
Decreased by 62.5%


Ratings contains information as such as each user rated to the movies

In [97]:
movies_data=pd.read_csv('movies_metadata.csv')

# 2. Data Pre-Processing

# 2.1 Data Cleaning

Displaying head of all dataframes

In [98]:
movies_data.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [99]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [100]:
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [101]:
keywords.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [102]:
credits.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [103]:
movies_data.shape

(45466, 24)

As dataset is huge, time taken will gradually increase let's reduce in a way that movies that should exist after year "2000"

In [104]:
movies_data['release_date'] = pd.to_datetime(movies_data['release_date'], errors='coerce')

In [105]:
def isBeforeGiven(givenYear, year=2003):
    if pd.isna(givenYear):  # Check if the value is NaT
        return False  # If it's NaT, return False
    convertedYear = int(str(givenYear)[:4])  # Convert the year to an integer
    return convertedYear >= year


In [106]:
movies=movies_data[movies_data['release_date'].apply(isBeforeGiven)]

In [107]:
movies.shape

(21448, 24)

As we combine all three datasets such as credits, keywords, movies_metadata by id. We need to ensure all three ids of one type so check the id type and convert

In [108]:
#checking the id of credits
type(credits.loc[0,'id'])

numpy.int32

In [109]:
#checking the id of keywords
type(keywords.loc[0,'id'])

numpy.int32

In [113]:
movies['id']=movies['id'].astype("int32")

so convert the datatype of movies_data into int32 to merge all three datasets

In [114]:
movies.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
868,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring a...,...,2010-05-25,0.0,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,NaN,Venice,False,7.5,4.0
1081,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0
2114,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",...,2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0
2564,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0
2778,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",NaN,171982,tt2378428,en,Romance,She's as hot as Britney Spears. Hotter. She pa...,...,2012-10-09,0.0,27.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Romance,False,0.0,0.0


In [115]:
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
868,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring a...,...,2010-05-25,0.0,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,NaN,Venice,False,7.5,4.0
1081,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0
2114,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",...,2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0
2564,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0
2778,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",NaN,171982,tt2378428,en,Romance,She's as hot as Britney Spears. Hotter. She pa...,...,2012-10-09,0.0,27.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Romance,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45453,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,404604,tt5690142,hi,Maa,The bliss of a biology teacher’s family life i...,...,2017-07-07,0.0,146.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mom,False,6.6,14.0
45454,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,420346,tt4130180,en,The Morning After,The Morning After is a feature film that consi...,...,2015-01-11,0.0,79.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,What happened last night?,The Morning After,False,4.0,2.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0


Now Let's Merge all datasets i.e., credits, keywords, movies

In [116]:
movies=pd.merge(movies,credits,on='id',how="inner").merge(keywords,on="id",how="inner")

In [117]:
movies.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring a...,...,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,NaN,Venice,False,7.5,4.0,"[{'cast_id': 1005, 'character': 'Marek', 'cred...","[{'credit_id': '52fe49e5c3a368484e145fb7', 'de...",[]
1,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,[],Released,NaN,The Sleepover,False,8.0,1.0,"[{'cast_id': 2, 'character': 'Rachel', 'credit...","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[]
2,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0,"[{'cast_id': 10, 'character': 'The Auctioneer'...","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...","[{'id': 214549, 'name': 'short'}]"
3,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"
4,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"


now our dataset contains of 27 attributes now Let's  clean the datset

name of id is simple. Changing into the same name as ratings dataset and helpful for future purposes.

In [118]:
movies.rename(columns={'id': 'movieId'}, inplace=True)

In [119]:
movies.head(5)

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring a...,...,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,NaN,Venice,False,7.5,4.0,"[{'cast_id': 1005, 'character': 'Marek', 'cred...","[{'credit_id': '52fe49e5c3a368484e145fb7', 'de...",[]
1,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,[],Released,NaN,The Sleepover,False,8.0,1.0,"[{'cast_id': 2, 'character': 'Rachel', 'credit...","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[]
2,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0,"[{'cast_id': 10, 'character': 'The Auctioneer'...","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...","[{'id': 214549, 'name': 'short'}]"
3,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"
4,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"


In [120]:
#movie dataset columns
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'movieId', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [121]:
#Interpreting the null values in the movie dataset
movies.isnull().sum()

adult                        0
belongs_to_collection    20189
budget                       0
genres                       0
homepage                 14989
movieId                      0
imdb_id                      9
original_language            5
original_title               0
overview                   501
popularity                   0
poster_path                154
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    139
spoken_languages             0
status                      42
tagline                  13133
title                        0
video                        0
vote_average                 0
vote_count                   0
cast                         0
crew                         0
keywords                     0
dtype: int64

In [122]:
# information about the movies_dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22176 entries, 0 to 22175
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  22176 non-null  object        
 1   belongs_to_collection  1987 non-null   object        
 2   budget                 22176 non-null  object        
 3   genres                 22176 non-null  object        
 4   homepage               7187 non-null   object        
 5   movieId                22176 non-null  int32         
 6   imdb_id                22167 non-null  object        
 7   original_language      22171 non-null  object        
 8   original_title         22176 non-null  object        
 9   overview               21675 non-null  object        
 10  popularity             22176 non-null  object        
 11  poster_path            22022 non-null  object        
 12  production_companies   22176 non-null  object        
 13  p

create a datframe "meta_data" that stores the unwanted information from the movies_dataset

In [123]:
meta_data=pd.DataFrame(data=movies['belongs_to_collection'])

In [124]:
meta_data['budget']= movies['budget']
meta_data['homepage']= movies['homepage']
meta_data['movieId']= movies['movieId']
meta_data['imdb_id']= movies['imdb_id']
meta_data['release_date	']= movies['release_date']
meta_data['revenue']= movies['revenue']
meta_data['vote_average']=movies['vote_average']
meta_data['vote_count']=movies['vote_count']
meta_data['runtime']=movies['runtime']
meta_data['poster_path']=movies['poster_path']
meta_data['popularity']=movies['popularity']

In [125]:
meta_data.head(5)

,belongs_to_collection,budget,homepage,movieId,imdb_id,release_date\t,revenue,vote_average,vote_count,runtime,poster_path,popularity
0,NaN,0,NaN,79782,tt1684935,2010-05-25,0.0,7.5,4.0,110.0,/lUmJiBTKdesFDkgSvV9zecCgNO6.jpg,0.14713
1,NaN,0,NaN,141210,tt2250194,2013-10-12,0.0,8.0,1.0,6.0,/pQpyEmFTGaox4yLuUXmiD2IDIbA.jpg,0.135596
2,NaN,0,http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,2012-06-20,0.0,10.0,1.0,18.0,/ePPNVWyIKYBdsGrOrYVaPKM8DlM.jpg,0.211754
3,NaN,0,NaN,84198,tt1736049,2012-03-22,0.0,6.9,7.0,84.0,/jn8L1QdWWX5c0NUOLjzaSXtZrbt.jpg,0.501046
4,NaN,0,NaN,84198,tt1736049,2012-03-22,0.0,6.9,7.0,84.0,/jn8L1QdWWX5c0NUOLjzaSXtZrbt.jpg,0.501046


In [126]:
movies.drop(columns=['belongs_to_collection','budget','poster_path','homepage','revenue','imdb_id','status','video','vote_average','vote_count'],inplace=True)

In [127]:
movies.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.14713,[],[],2010-05-25,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",NaN,Venice,"[{'cast_id': 1005, 'character': 'Marek', 'cred...","[{'credit_id': '52fe49e5c3a368484e145fb7', 'de...",[]
1,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,[],[],2013-10-12,6.0,[],NaN,The Sleepover,"[{'cast_id': 2, 'character': 'Rachel', 'credit...","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[]
2,False,"[{'id': 18, 'name': 'Drama'}]",143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,[],[],2012-06-20,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,The Farmer's Wife,"[{'cast_id': 10, 'character': 'The Auctioneer'...","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...","[{'id': 214549, 'name': 'short'}]"
3,False,"[{'id': 99, 'name': 'Documentary'}]",84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-22,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",One Nation. Underfed.,A Place at the Table,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"
4,False,"[{'id': 99, 'name': 'Documentary'}]",84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-22,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",One Nation. Underfed.,A Place at the Table,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"


In [128]:
#calculating the null values in columns
movies.isnull().sum()

adult                       0
genres                      0
movieId                     0
original_language           5
original_title              0
overview                  501
popularity                  0
production_companies        0
production_countries        0
release_date                0
runtime                   139
spoken_languages            0
tagline                 13133
title                       0
cast                        0
crew                        0
keywords                    0
dtype: int64

In [129]:
#shape of movies_dataset
movies.shape

(22176, 17)

In [130]:
movies['overview']=movies['overview'].fillna('a story')  
movies['original_language']=movies['original_language'].fillna('en')
#movies['production_companies']=movies['production_companies'].fillna('independent production')
#movies['production_countries']=movies['production_countries'].fillna('United States')
#movies['spoken_languages']=movies['spoken_languages'].fillna('en')
#movies['tagline']=movies['tagline'].fillna('')

In [131]:
movies.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.14713,[],[],2010-05-25,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",NaN,Venice,"[{'cast_id': 1005, 'character': 'Marek', 'cred...","[{'credit_id': '52fe49e5c3a368484e145fb7', 'de...",[]
1,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,[],[],2013-10-12,6.0,[],NaN,The Sleepover,"[{'cast_id': 2, 'character': 'Rachel', 'credit...","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[]
2,False,"[{'id': 18, 'name': 'Drama'}]",143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,[],[],2012-06-20,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,The Farmer's Wife,"[{'cast_id': 10, 'character': 'The Auctioneer'...","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...","[{'id': 214549, 'name': 'short'}]"
3,False,"[{'id': 99, 'name': 'Documentary'}]",84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-22,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",One Nation. Underfed.,A Place at the Table,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"
4,False,"[{'id': 99, 'name': 'Documentary'}]",84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-22,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",One Nation. Underfed.,A Place at the Table,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de...","[{'id': 187056, 'name': 'woman director'}]"


In [132]:
movies.shape

(22176, 17)

In [133]:
#calculating the null values in columns
movies.isnull().sum()

adult                       0
genres                      0
movieId                     0
original_language           0
original_title              0
overview                    0
popularity                  0
production_companies        0
production_countries        0
release_date                0
runtime                   139
spoken_languages            0
tagline                 13133
title                       0
cast                        0
crew                        0
keywords                    0
dtype: int64

In [134]:
runtime=movies['runtime'].mean()
runtime

93.8734855016563

In [135]:
movies["runtime"].fillna(runtime, inplace = True)

In [136]:
movies.isnull().sum()

adult                       0
genres                      0
movieId                     0
original_language           0
original_title              0
overview                    0
popularity                  0
production_companies        0
production_countries        0
release_date                0
runtime                     0
spoken_languages            0
tagline                 13133
title                       0
cast                        0
crew                        0
keywords                    0
dtype: int64

# 2.2 corrrecting the data-format

In [137]:
#handling the genres 
movies.loc[0,'genres']

"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"

In [138]:
#function to correct the format for the genres as it is in dictionary format
def correct_Genres(data):
    result = ''.join(filter(lambda x: x.isalpha(), data))
    result = result.replace("id","")
    result = result.replace("name","")
    return result

In [139]:
#applying the function to movie-genres columns
movies['genres']=movies['genres'].apply(correct_Genres)

In [140]:
#checking the values
movies.loc[0,'genres']

'DramaRomance'

In [141]:
 #correcting the format for production companies attribute
movies.loc[0,'production_companies']   

'[]'

In [142]:
def correct_production_companies(data):
    result = ''.join(filter(lambda x: x.isalpha(), data))
    result = result.replace("id","")
    result = result.replace("name","")
    return result

In [143]:
movies['production_companies']=movies['production_companies'].apply(correct_production_companies)

In [144]:
movies.loc[0,'production_companies']  

''

In [145]:
#handling the production_countries
movies.loc[707,'production_countries']

"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [146]:
def correct_production_countries(data):
    result = ''.join(filter(lambda x: x.isalpha(), data))
    result = result.replace("iso","")
    result = result.replace("name","")
    return result

In [147]:
movies['production_countries']=movies['production_countries'].apply(correct_production_countries)

In [148]:
movies.loc[707,'production_countries']

'CACanadaUSUnitedStatesofAmerica'

In [149]:
#handling the languages spoken
movies.loc[707,'spoken_languages']

"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'ja', 'name': '日本語'}]"

In [150]:
def correct_spoken_languages(data):
    result = ''.join(filter(lambda x: x.isalpha(), data))
    result = result.replace("iso","")
    result = result.replace("name","")
    return result

In [151]:
movies['spoken_languages']=movies['spoken_languages'].apply(correct_spoken_languages)

In [152]:
movies.loc[707,'spoken_languages']

'enEnglishja日本語'

In [153]:
#handling the cast_columns

In [154]:
movies.loc[0,"cast"]

'[{\'cast_id\': 1005, \'character\': \'Marek\', \'credit_id\': \'52fe49e5c3a368484e145fcd\', \'gender\': 0, \'id\': 587975, \'name\': \'Marcin Walewski\', \'order\': 0, \'profile_path\': None}, {\'cast_id\': 1007, \'character\': "Joanna - Marek\'s mother", \'credit_id\': \'5432c0680e0a2656e2004b57\', \'gender\': 0, \'id\': 591258, \'name\': \'Magdalena Cielecka\', \'order\': 1, \'profile_path\': \'/tYQNfa9WhK39Y8AXCDzs6RQK5Qw.jpg\'}, {\'cast_id\': 1008, \'character\': "Roman - Marek\'s father", \'credit_id\': \'5432c081c3a36831a9005dc2\', \'gender\': 0, \'id\': 140221, \'name\': \'Mariusz Bonaszewski\', \'order\': 2, \'profile_path\': \'/opB6JZyMD4aIOVSB1c8o2BQjwOv.jpg\'}, {\'cast_id\': 1009, \'character\': \'Barbara - Joannas sister\', \'credit_id\': \'5432c094c3a3681143004b22\', \'gender\': 1, \'id\': 105562, \'name\': \'Agnieszka Grochowska\', \'order\': 3, \'profile_path\': \'/2apU86P6peRMxzNRZJV1sYl8X8B.jpg\'}, {\'cast_id\': 1010, \'character\': \'Weronika - Joannas sister\', \'cr

In [155]:
def correct_cast(data):
    # Example string representing a list of dictionaries
    # Convert the string to a list of dictionaries
    
    data=ast.literal_eval(data)    # Use ast.literal_eval to safely evaluate the string into a Python object
    
    json_string = json.dumps(data)   # Now convert the list of dictionaries to a JSON-formatted string
    my_list = json.loads(json_string)

    keys_to_remove=['cast_id','character','credit_id','gender','id','order','profile_path']

    for diction in my_list:
        for key in keys_to_remove:
            diction.pop(key)
    result=[]
    
    for diction in my_list:
        result.append(diction['name'])

    cast="".join(result)
    return cast

In [156]:
movies['cast']=movies['cast'].apply(correct_cast)

In [157]:
movies.loc[0,'cast']

'Marcin WalewskiMagdalena CieleckaMariusz BonaszewskiAgnieszka GrochowskaGrażyna Błęcka-KolskaJulia Kijowska'

In [158]:
#handling the crew data

In [159]:
movies.loc[0,"crew"]

"[{'credit_id': '52fe49e5c3a368484e145fb7', 'department': 'Directing', 'gender': 0, 'id': 587971, 'job': 'Director', 'name': 'Jan Jakub Kolski', 'profile_path': None}, {'credit_id': '52fe49e5c3a368484e145fc3', 'department': 'Crew', 'gender': 2, 'id': 6427, 'job': 'Cinematography', 'name': 'Artur Reinhart', 'profile_path': None}, {'credit_id': '52fe49e5c3a368484e145fc9', 'department': 'Editing', 'gender': 0, 'id': 587974, 'job': 'Editor', 'name': 'Witold Chominski', 'profile_path': None}, {'credit_id': '571c8b2e92514115e200242b', 'department': 'Production', 'gender': 0, 'id': 1244217, 'job': 'Producer', 'name': 'Michał Kwieciński', 'profile_path': None}, {'credit_id': '5432c0470e0a265872004aad', 'department': 'Writing', 'gender': 0, 'id': 587971, 'job': 'Writer', 'name': 'Jan Jakub Kolski', 'profile_path': None}]"

In [160]:
def correct_crew(data):
    # Example string representing a list of dictionaries
    # Convert the string to a list of dictionaries
    
    data=ast.literal_eval(data)    # Use ast.literal_eval to safely evaluate the string into a Python object
    
    json_string = json.dumps(data)   # Now convert the list of dictionaries to a JSON-formatted string
    my_list = json.loads(json_string)

    keys_to_remove=['credit_id','department','job','gender','id','profile_path']

    for diction in my_list:
        for key in keys_to_remove:
            diction.pop(key)
    result=[]
    
    for diction in my_list:
        result.append(diction['name'])

    crew="".join(result)
    return crew

In [161]:
movies['crew']=movies['crew'].apply(correct_crew)

In [162]:
movies.loc[0,'crew']

'Jan Jakub KolskiArtur ReinhartWitold ChominskiMichał KwiecińskiJan Jakub Kolski'

In [163]:
#handling the keywords

In [164]:
movies.loc[707,'keywords']

"[{'id': 779, 'name': 'martial arts'}, {'id': 9717, 'name': 'based on comic'}, {'id': 12367, 'name': 'female assassin'}, {'id': 15285, 'name': 'spin off'}]"

In [165]:
def correct_keywords(data):
    # Example string representing a list of dictionaries
    # Convert the string to a list of dictionaries
    
    data=ast.literal_eval(data)    # Use ast.literal_eval to safely evaluate the string into a Python object
    
    json_string = json.dumps(data)   # Now convert the list of dictionaries to a JSON-formatted string
    my_list = json.loads(json_string)

    keys_to_remove=['id']

    for diction in my_list:
        for key in keys_to_remove:
            diction.pop(key)
    result=[]
    
    for diction in my_list:
        result.append(diction['name'])

    key_words="".join(result)
    return key_words

In [166]:
movies['keywords']=movies['keywords'].apply(correct_keywords)

In [167]:
movies.loc[707,'keywords']

'martial artsbased on comicfemale assassinspin off'

In [168]:
movies.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,DramaRomance,79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.14713,,,2010-05-25,110.0,plPolski,NaN,Venice,Marcin WalewskiMagdalena CieleckaMariusz Bonas...,Jan Jakub KolskiArtur ReinhartWitold Chominski...,
1,False,ComedyHorror,141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,,,2013-10-12,6.0,,NaN,The Sleepover,Josh FeldmanGus KampCarolyn JaniaKristine Blac...,Chris Cullari,
2,False,Drama,143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,,,2012-06-20,18.0,enEnglish,NaN,The Farmer's Wife,James CartwrightGeraldine JamesAlex KellyMarti...,John ParishJonas GrimåsGary ShawFrancis LeeFra...,short
3,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director
4,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director


Among all the attributes, we can say there spoken languages, keywords, cast,crew,overview are important as recommendation system has influenced by them

In [169]:
def fill_languages(data):
    if data=="":
        return "English"
    else:
        return data


In [170]:
movies['spoken_languages']=movies['spoken_languages'].apply(fill_languages)

In [171]:
movies.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,DramaRomance,79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.14713,,,2010-05-25,110.0,plPolski,NaN,Venice,Marcin WalewskiMagdalena CieleckaMariusz Bonas...,Jan Jakub KolskiArtur ReinhartWitold Chominski...,
1,False,ComedyHorror,141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,,,2013-10-12,6.0,English,NaN,The Sleepover,Josh FeldmanGus KampCarolyn JaniaKristine Blac...,Chris Cullari,
2,False,Drama,143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,,,2012-06-20,18.0,enEnglish,NaN,The Farmer's Wife,James CartwrightGeraldine JamesAlex KellyMarti...,John ParishJonas GrimåsGary ShawFrancis LeeFra...,short
3,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director
4,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director


In [172]:
type(movies.loc[2,'spoken_languages'])

str

In [173]:
movies.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,DramaRomance,79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.14713,,,2010-05-25,110.0,plPolski,NaN,Venice,Marcin WalewskiMagdalena CieleckaMariusz Bonas...,Jan Jakub KolskiArtur ReinhartWitold Chominski...,
1,False,ComedyHorror,141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,,,2013-10-12,6.0,English,NaN,The Sleepover,Josh FeldmanGus KampCarolyn JaniaKristine Blac...,Chris Cullari,
2,False,Drama,143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,,,2012-06-20,18.0,enEnglish,NaN,The Farmer's Wife,James CartwrightGeraldine JamesAlex KellyMarti...,John ParishJonas GrimåsGary ShawFrancis LeeFra...,short
3,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director
4,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director


In [174]:
def fill_keywords(data):
    if data=="":
        return "English"
    else:
        return data

In [175]:
movies['keywords']=movies['keywords'].apply(fill_keywords)

In [176]:
movies.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,DramaRomance,79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.14713,,,2010-05-25,110.0,plPolski,NaN,Venice,Marcin WalewskiMagdalena CieleckaMariusz Bonas...,Jan Jakub KolskiArtur ReinhartWitold Chominski...,English
1,False,ComedyHorror,141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,,,2013-10-12,6.0,English,NaN,The Sleepover,Josh FeldmanGus KampCarolyn JaniaKristine Blac...,Chris Cullari,English
2,False,Drama,143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,,,2012-06-20,18.0,enEnglish,NaN,The Farmer's Wife,James CartwrightGeraldine JamesAlex KellyMarti...,John ParishJonas GrimåsGary ShawFrancis LeeFra...,short
3,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director
4,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director


In [177]:
import os

# Specify the directory path
directory = '/mood based/hybrid-recommendation/'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save the DataFrame to CSV
movies.to_csv(os.path.join(directory, 'output.csv'), index=False)


In [178]:
movies.to_csv('output.csv',index=False)